<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/Working_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
! apt-get install libboost-python1.71-dev openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make FILES
!make -j4 OBJECTS
!make LINKAGE
!ls *.so

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-python1.71-dev is already the newest version (1.71.0-6ubuntu6).
libboost-python1.71-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu1.5).
zlib1g-dev set to manually installed.
The following additional packages will be installed:
  libgsl23 libgslcblas0 libnlopt-cxx0 libnlopt-dev libnlopt0 libopenbabel6
  libschroedinger-maeparser1 python-is-python3
Suggested packages:
  libeigen3-doc libmpfrc++-dev gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info
  | gsl-ref-html libopenbabel-doc
The following NEW packages will be installed:
  libeigen3-dev libgsl-dev libgsl23 libgslcblas0 libnlopt-cxx-dev
  libnlopt-cxx0 libnlopt-dev libnlopt0 libopenbabel-dev libopenbabel6
  libschroedinger-maeparser1 openbabel python-dev-is-python3 python-is-python3
0 upgraded, 14 newly installed, 0 to remove and 24 not upgraded.
Need to get 7,174 kB of archives.
After this

In [2]:
!cat src/pyOptimizer.cpp

/*
 * Optimizer.cpp
 *
 *  Created on: 21/03/2012
 *      Author: Nascimento
 */

#include "pyOptimizer.h"

Optimizer::Optimizer(Mol2* _Rec, Mol2* _RefLig, PARSER* _Parser) {
    this->RefLig = _RefLig;
    this->Rec = _Rec;
    this->Parser = _Parser;
}
Optimizer::Optimizer(Mol2* _Rec, Mol2* _RefLig, PARSER* _Parser, Grid* _Grids) {
    this->RefLig = _RefLig;
    this->Rec = _Rec;
    this->Parser = _Parser;
    this->Grids = _Grids;
}

Optimizer::~Optimizer() {
}

double Optimizer::distance(vector<double> atom1, vector<double> atom2) {
    return ( sqrt(((atom2[0]-atom1[0])*(atom2[0]-atom1[0]))+((atom2[1]-atom1[1])*(atom2[1]-atom1[1]))+((atom2[2]-atom1[2])*(atom2[2]-atom1[2]))) );
}
double Optimizer::distance_squared(vector<double> atom1, vector<double> atom2) {
    return ( (((atom2[0]-atom1[0])*(atom2[0]-atom1[0]))+((atom2[1]-atom1[1])*(atom2[1]-atom1[1]))+((atom2[2]-atom1[2])*(atom2[2]-atom1[2]))) );
}

vector<vector<double> > Optimizer::update_coords(const std::vector<double> &x

In [ ]:
from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyMcEntropy import *
from pySA import *
from pyOptimizer import *


In [3]:
!ldd pyOptimizer.so

	linux-vdso.so.1 (0x00007ffe076fa000)
	libboost_python38.so.1.71.0 => /lib/x86_64-linux-gnu/libboost_python38.so.1.71.0 (0x00007f1388b01000)
	libz.so.1 => /lib/x86_64-linux-gnu/libz.so.1 (0x00007f1388ae5000)
	libopenbabel.so.6 => /lib/x86_64-linux-gnu/libopenbabel.so.6 (0x00007f1388889000)
	libgsl.so.23 => /lib/x86_64-linux-gnu/libgsl.so.23 (0x00007f138860d000)
	libnlopt_cxx.so.0 => /lib/x86_64-linux-gnu/libnlopt_cxx.so.0 (0x00007f1388592000)
	libstdc++.so.6 => /lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007f13883b0000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f138825f000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007f1388244000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f1388052000)
	libpthread.so.0 => /lib/x86_64-linux-gnu/libpthread.so.0 (0x00007f138802f000)
	libdl.so.2 => /lib/x86_64-linux-gnu/libdl.so.2 (0x00007f1388029000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f1388d46000)
	libgomp.so.1 => /lib/x86_64-linux-gnu/libgomp.so.1 (0x00007f1387fe70

In [ ]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 30.0 #@param {type:"number"}
Input.search_box_y = 30.0 #@param {type:"number"}
Input.search_box_z = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.3 #@param {type:"number"}
Input.x_dim = 8.0 #@param {type:"number"}
Input.y_dim = 8.0 #@param {type:"number"}
Input.z_dim = 8.0 #@param {type:"number"}
Input.write_grids = True;

In [ ]:

Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
lig = Mol2(Input, "test/lig.mol2.gz")
rec = Mol2(Input, "test/rec.mol2.gz")
Energy = Energy2(Input)

for i in range(len(list(rec.residue_pointer))-1):
  HB.parse_residue(rec.residue_pointer[i]-1, rec.residue_pointer[i+1]-2, rec.resnames[i], rec, lig, 9.0)

HB.find_ligandHB("test/lig.mol2.gz", lig)

com = Coord.compute_com(lig)

print(list(lig.xyz))

energy1 = Energy.compute_ene(rec, lig, lig.xyz)
print("Original energy: %7.3f" % energy1)

Input.write_grids = True;
Grids = Grid(Input, Writer, rec, com)

energy2 = Energy.compute_ene(Grids, lig, lig.xyz)
print("Grid energy: %7.3f" % energy2)